dans ce fichier nous allons mettre les requêtes de filtres et nettoyage des données

Import pandas

In [ ]:
import pandas as pd

Import des données : tables entières

In [ ]:
tmdb = pd.read_csv("./Data/TMDB/tmdb_full.csv", low_memory=False)

In [ ]:
title_akas = pd.read_csv("./Data/IMDb/title.akas.tsv/title.akas.tsv", sep="\t", low_memory=False)

In [ ]:
title_basics = pd.read_csv("./Data/IMDb/title.basics.tsv/title.basics.tsv", sep="\t", low_memory=False)

In [ ]:
title_crew = pd.read_csv("./Data/IMDb/title.crew.tsv/title.crew.tsv", sep="\t", low_memory=False)

In [ ]:
title_episode = pd.read_csv("./Data/IMDb/title.episode.tsv/title.episode.tsv", sep="\t", low_memory=False)

In [ ]:
title_principals = pd.read_csv("./Data/IMDb/title.principals.tsv/title.principals.tsv", sep="\t", low_memory=False)

In [ ]:
title_ratings = pd.read_csv("./Data/IMDb/title.ratings.tsv/title.ratings.tsv", sep="\t", low_memory=False)

In [ ]:
name_basics = pd.read_csv("./Data/IMDb/name.basics.tsv/name.basics.tsv", sep="\t", low_memory=False)

Filtre par genre = Action et production countries = FR

In [ ]:
tmdb_filtre = tmdb[(tmdb['genres'].str.contains('Action'))|(tmdb['production_countries'].str.contains('FR'))]